# 4-1 Attrition Trends and Patterns

In [ ]:
# Gives attrition rate of one sub-category:
# Columns = a list of columns we want to get calculations for their sub-categories 
# (for ex., if we write ['Gender' we'll get results for female,male])
# MinEmployeed = filters out groups with less employees.
#RowstoPresent = shows top X results.

def AttritionRateCat(Columns, MinEmployees, RowstoPresent):
    rows = []

    for Column in Columns:
        Categories = 'HRDB_cleaned.csv'[Column].unique().tolist()

        for cat in Categories:
            mask = 'HRDB_cleaned.csv'['HRDB_cleaned.csv'[Column]== cat]
            count = len(mask)
            if count < MinEmployees:
                continue
        
            rate = round(100*(mask['Attrition']=='Yes').mean(),2)
        
            rows.append({
                'Category': Column,
                'Sub-Category': cat,
                'NumberofEmployees': count,
                'AttritionRate': rate
            })

    return pd.DataFrame(rows).sort_values(by = 'AttritionRate',ascending = False).head(RowstoPresent)




In [4]:
# This variable returns a list of all columns' headers that are categorial or an object
AllCols = 'HRDB_cleaned.csv'.select_dtypes(include=['object', 'category']).columns.tolist()

AttritionRateCat(AllCols,10,5)

AttributeError: 'str' object has no attribute 'select_dtypes'